# Chapter 4: Text Classification - Hard Tasks

This notebook tackles advanced classification challenges. You'll implement hierarchical multi-level classifiers for complex taxonomies, use active learning to minimize labeling costs, build ensemble classifiers to improve robustness, and apply transfer learning across domains. These techniques are essential for production-level NLP systems.


## Setup

Run all cells in this section to set up the environment and load necessary data.


### [Optional] - Installing Packages on <img src="https://colab.google/static/images/icons/colab.png" width=100>

If you are viewing this notebook on Google Colab (or any other cloud vendor), you need to **uncomment and run** the following codeblock to install the dependencies for this chapter:

 **Note**: We will want to use a GPU to run the examples in this notebook. In Google Colab, go to
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---


In [ ]:
 %%capture
!pip install transformers sentence-transformers openai
!pip install -U datasets

### Data Loading


In [ ]:
from datasets import load_dataset
# Load our data
data = load_dataset("rotten_tomatoes")
data

### Helper Functions


In [ ]:
from sklearn.metrics import classification_report
def evaluate_performance(y_true, y_pred):
    """Create and print the classification report"""
    performance = classification_report(
        y_true, y_pred,
        target_names=["Negative Review", "Positive Review"]
    )
    print(performance)

## Your Turn - Text Classification Experiments

Run each task first to see the baseline results. Follow the instructions to modify and experiment.

This section is divided into EASY, MEDIUM, & HARD.

## Hard Tasks


### Hard Tasks - Advanced Classification Challenges

These tasks require significant modifications and deeper understanding. Take your time and experiment

#### Hard Task 1: Hierarchical Multi-Level Classifier
Instead of flat classification (choosing from all categories at once), hierarchical classification makes decisions in steps: first broad categories, then fine-grained ones. This mirrors how humans often reason.
Instructions:
1. Run the 2-level classifier (Sentiment → Specific Aspect)
2. Compare with flat classification to see confidence differences
3. Try adding a 3rd level by uncommenting the code
4. Analyze whether breaking decisions into steps helps or hurts

In [ ]:
# Level 1: Broad sentiment
level1_labels = [
    "negative sentiment review",
    "positive sentiment review"
]
# Level 2: Specific aspects (conditional on Level 1)
level2_negative = [
    "review criticizing entertainment value and pacing",
    "review criticizing technical quality and production"
]
level2_positive = [
    "review praising technical quality and artistry",
    "review praising entertainment value and enjoyment"
]
# Add Level 3 for even finer granularity
# level3_positive_quality = [...]
# level3_positive_entertainment = [...]

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

test_reviews = [
    "Amazing cinematography and brilliant direction!",
    "Terrible pacing, very boring throughout",
    "Excellent acting but weak storyline",
    "Poor production quality, disappointing visuals"
]

In [ ]:
def hierarchical_classify_2level(review):
    """
    Classify review using 2-level hierarchy:
    Level 1: Sentiment (positive/negative)
    Level 2: Specific aspect (quality/entertainment)
    """
    # Level 1: Determine sentiment
    level1_emb = model.encode(level1_labels)
    review_emb = model.encode([review])
    level1_sim = cosine_similarity(review_emb, level1_emb)[0]

    level1_pred = np.argmax(level1_sim)
    level1_conf = level1_sim[level1_pred]
    level1_label = level1_labels[level1_pred]

    # Level 2: Conditional on Level 1
    if level1_pred == 0:  # Negative
        level2_labels = level2_negative
        path = "Negative → "
    else:  # Positive
        level2_labels = level2_positive
        path = "Positive → "

    level2_emb = model.encode(level2_labels)
    level2_sim = cosine_similarity(review_emb, level2_emb)[0]

    level2_pred = np.argmax(level2_sim)
    level2_conf = level2_sim[level2_pred]
    level2_label = level2_labels[level2_pred]
    path += level2_label

    return {
        "level1_label": level1_label,
        "level1_conf": level1_conf,
        "level2_label": level2_label,
        "level2_conf": level2_conf,
        "path": path
    }

Classify the reviews:


In [ ]:
print("Hierarchical Classification (2 LEVELS)")
for i, review in enumerate(test_reviews):
    result = hierarchical_classify_2level(review)
    print(f"\nReview {i+1}: '{review}'")
    print(f"\n  Level 1 (Sentiment):")
    print(f"     {result['level1_label']}")
    print(f"     Confidence: {result['level1_conf']:.3f}")
    print(f"\n  Level 2 (Specific Aspect):")
    print(f"     {result['level2_label']}")
    print(f"     Confidence: {result['level2_conf']:.3f}")
    print(f"\n  Final Classification Path:")
    print(f"     {result['path']}")
# Compare with flat classification
print()
print("Comparison: Hierarchical vs Flat Classification")
# Flat: All 4 categories at once
flat_labels = [
    "review criticizing entertainment value and pacing",      # 0
    "review criticizing technical quality and production",    # 1
    "review praising technical quality and artistry",         # 2
    "review praising entertainment value and enjoyment"       # 3
]
flat_embeddings = model.encode(flat_labels)
review_embeddings = model.encode(test_reviews)
flat_sim = cosine_similarity(review_embeddings, flat_embeddings)
print("\nShowing first 3 reviews:")
for i in range(min(3, len(test_reviews))):
    hier_result = hierarchical_classify_2level(test_reviews[i])
    flat_pred = np.argmax(flat_sim[i])
    flat_conf = flat_sim[i][flat_pred]
    print(f"\nReview: '{test_reviews[i][:50]}...'")
    print(f"  Hierarchical: {hier_result['level2_label']}")
    print(f"     Confidence: {hier_result['level2_conf']:.3f}")
    print(f"  Flat:         {flat_labels[flat_pred]}")
    print(f"     Confidence: {flat_conf:.3f}")
    print(f"  Confidence Diff: {hier_result['level2_conf'] - flat_conf:+.3f}")
# After implementing 3-level, uncomment to test it:
# print()
# print("Testing 3-LEVEL Hierarchical Classification")
# for i, review in enumerate(test_reviews):
#     result = hierarchical_classify_3level(review)
#     print(f"\n{i+1}. '{review[:60]}...'")
#     print(f"   Path: {result['path']}")
#     print(f"   Level 3 confidence: {result['level3_conf']:.3f}")

The hierarchical approach makes decisions in stages:
1. Level 1: Determines if the review is positive or negative
2. Level 2: Based on that sentiment, classifies the specific aspect

Notice the confidence scores at each level. The hierarchical classifier's final confidence is often higher than flat classification because it's making simpler decisions at each step.

Looking at the comparison between hierarchical and flat approaches:
- Hierarchical often has higher confidence (makes easier per-step decisions)
- But if Level 1 is wrong, Level 2 has no chance to correct it
- Flat classification considers all options at once but may be less confident

#### Hard Task 2: Active Learning to Minimize Labeling Costs
Labeling data is expensive. Active learning strategically selects the most informative samples to label, potentially saving 50%+ of labeling effort compared to random selection. Run the simulation to see active learning vs random sampling, observe which samples the model finds "uncertain", and track how many samples each approach needs to reach F1=0.85. Try implementing the alternative selection strategy (margin sampling) and compare labeling cost savings.

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

# Simulate active learning vs random sampling
print("Active Learning Simulation")
print("="*60)

# Setup
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
train_data = data["train"].shuffle(seed=42).select(range(1000))
test_data = data["test"].shuffle(seed=42).select(range(200))

# Start with small labeled set
labeled_size = 50
target_f1 = 0.85

print(f"Starting with {labeled_size} labeled samples")
print(f"Target F1: {target_f1}")
print(f"\nNote: This is a simplified simulation")
print("In real active learning, you'd query an oracle (human) for labels\n")

In [ ]:
# Encode all training and test data upfront
all_train_embeddings = model.encode(train_data["text"], show_progress_bar=False)
all_test_embeddings = model.encode(test_data["text"], show_progress_bar=False)

print(f"Encoded {len(train_data)} training samples")
print(f"Encoded {len(test_data)} test samples")

Start with a small labeled set and keep rest unlabeled:

In [ ]:
labeled_indices = list(range(labeled_size))  # First 50 samples
unlabeled_indices = list(range(labeled_size, len(train_data)))  # Rest are unlabeled

print(f"Starting labeled set: {len(labeled_indices)} samples")
print(f"Unlabeled pool: {len(unlabeled_indices)} samples")

Run active learning iterations:

In [ ]:
print("\nIteration | Labeled Size | F1 Score")
print("-" * 40)

for iteration in range(5):
    # Train on current labeled data
    X_train = all_train_embeddings[labeled_indices]
    y_train = [train_data["label"][i] for i in labeled_indices]

    clf = LogisticRegression(random_state=42, max_iter=1000)
    clf.fit(X_train, y_train)

    # Evaluate on test set
    y_pred = clf.predict(all_test_embeddings)
    current_f1 = f1_score(test_data["label"], y_pred, average='weighted')

    print(f"{iteration:9d} | {len(labeled_indices):12d} | {current_f1:.4f}")

    if current_f1 >= target_f1:
        print(f"\nReached target F1={target_f1}!")
        break

    # Complete this: Select most uncertain samples from unlabeled pool
    if len(unlabeled_indices) > 0:
        unlabeled_embeddings = all_train_embeddings[unlabeled_indices]
        probs = clf.predict_proba(unlabeled_embeddings)

        # Calculate uncertainty for each unlabeled sample
        # Hint: Uncertainty = 1 - max(probability)
        uncertainties = None  # calculate this

        # Select 20 most uncertain samples
        # Hint: Use argsort to find indices of highest uncertainties
        n_select = min(20, len(unlabeled_indices))
        most_uncertain = None  # find n_select most uncertain indices
        selected_indices = [unlabeled_indices[idx] for idx in most_uncertain] if most_uncertain is not None else []

        # Move to labeled set
        labeled_indices.extend(selected_indices)
        unlabeled_indices = [idx for idx in unlabeled_indices if idx not in selected_indices]

Active learning iteratively:
1. Trains on currently labeled data
2. Finds the most uncertain unlabeled samples
3. "Labels" those samples (adds them to training set)
4. Repeats

The uncertain samples being selected typically have prediction probabilities close to 50-50. These are the most informative because they lie near the decision boundary.

Compare the learning curves to see how quickly each approach improves. Active learning often reaches target performance with fewer labeled samples, saving significant labeling costs.

Experiment: Try margin sampling as an alternative selection strategy (selects samples where top 2 predictions are close):

In [ ]:
# Experiment: Alternative selection strategy - Margin Sampling
# Uncomment to try margin sampling instead of uncertainty sampling:
#
# # Reset for fair comparison
# labeled_indices = list(range(labeled_size))
# unlabeled_indices = list(range(labeled_size, len(train_data)))
#
# print("\nMargin Sampling Strategy:")
# print("-" * 40)
#
# for iteration in range(5):
#     X_train = all_train_embeddings[labeled_indices]
#     y_train = [train_data["label"][i] for i in labeled_indices]
#     
#     clf = LogisticRegression(random_state=42, max_iter=1000)
#     clf.fit(X_train, y_train)
#     
#     y_pred = clf.predict(all_test_embeddings)
#     current_f1 = f1_score(test_data["label"], y_pred, average='weighted')
#     
#     print(f"{iteration:9d} | {len(labeled_indices):12d} | {current_f1:.4f}")
#     
#     if current_f1 >= target_f1:
#         print(f"\nReached target F1={target_f1}!")
#         break
#     
#     if len(unlabeled_indices) > 0:
#         unlabeled_embeddings = all_train_embeddings[unlabeled_indices]
#         probs = clf.predict_proba(unlabeled_embeddings)
#         
#         # Margin = difference between top 2 probabilities
#         sorted_probs = np.sort(probs, axis=1)
#         margins = sorted_probs[:, -1] - sorted_probs[:, -2]  # Smaller margin = more uncertain
#         
#         n_select = min(20, len(unlabeled_indices))
#         smallest_margins = np.argsort(margins)[:n_select]  # Get smallest margins
#         selected_indices = [unlabeled_indices[idx] for idx in smallest_margins]
#         
#         labeled_indices.extend(selected_indices)
#         unlabeled_indices = [idx for idx in unlabeled_indices if idx not in selected_indices]
#
# print("\nCompare: Did margin sampling reach target faster or slower than uncertainty sampling?")

#### Hard Task 3: Ensemble Classifier for Improved Robustness

Ensemble methods combine multiple models to reduce individual biases and improve reliability. The wisdom of crowds principle: multiple imperfect models together often beat any single model.

Instructions:
1. Run to see 3 individual models compared to ensemble methods
2. Compare simple majority voting vs confidence-weighted voting
3. Examine disagreement cases - when models disagree, which is usually right?
4. Optionally add a 4th model and performance-weighted voting
5. Determine if the ensemble beats the best individual model

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

print("Ensemble CLASSIFIER Comparison")
print("="*60)

# Load 3 different models
model1 = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
model2 = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model3 = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L3-v2')

models = [model1, model2, model3]
model_names = ["MPNet", "MiniLM-L6", "MiniLM-L3"]

print(f"Using {len(models)} models:\n")
for name in model_names:
    print(f"  - {name}")

# Train each model
train_subset = data["train"].shuffle(seed=42).select(range(1000))
test_subset = data["test"].shuffle(seed=42).select(range(200))

print(f"\nTraining on {len(train_subset)} samples...")
print("Note: In practice, ensemble diversity comes from:")
print("  - Different model architectures")
print("  - Different training data subsets")
print("  - Different hyperparameters\n")

Train each model individually and track performance:

In [ ]:
classifiers = []
individual_f1s = []

print("Training individual models...\n")

In [ ]:
for i, (model, name) in enumerate(zip(models, model_names)):
    # Encode with this model
    train_emb = model.encode(train_subset["text"], show_progress_bar=False)
    test_emb = model.encode(test_subset["text"], show_progress_bar=False)

    # Train classifier
    clf = LogisticRegression(random_state=42, max_iter=1000)
    clf.fit(train_emb, train_subset["label"])

    # Evaluate
    y_pred = clf.predict(test_emb)
    f1 = f1_score(test_subset["label"], y_pred, average='weighted')

    classifiers.append((clf, test_emb))
    individual_f1s.append(f1)

    print(f"{name:12s}: F1 = {f1:.4f}")

Collect predictions from all models for ensemble voting:

In [ ]:
all_predictions = []

for clf, test_emb in classifiers:
    preds = clf.predict(test_emb)
    all_predictions.append(preds)

all_predictions = np.array(all_predictions)
print(f"\nCollected predictions from {len(all_predictions)} models")

Apply majority voting to get ensemble predictions:

In [ ]:
ensemble_predictions = []

for i in range(len(test_subset)):
    votes = all_predictions[:, i]
    unique, counts = np.unique(votes, return_counts=True)
    majority = unique[np.argmax(counts)]  # Pick most common vote
    ensemble_predictions.append(majority)

ensemble_f1 = f1_score(test_subset["label"], ensemble_predictions, average='weighted')
print(f"Ensemble F1: {ensemble_f1:.4f}")

Compare ensemble vs individual models:

In [ ]:
print("\nResults Comparison")
print("-" * 40)

for name, f1 in zip(model_names, individual_f1s):
    print(f"{name:12s}: {f1:.4f}")

print(f"\nEnsemble:     {ensemble_f1:.4f}")

best_individual = max(individual_f1s)
if ensemble_f1 > best_individual:
    gain = ensemble_f1 - best_individual
    print(f"\n✓ Ensemble wins by {gain:.4f}!")
else:
    diff = best_individual - ensemble_f1
    print(f"\nBest individual model wins by {diff:.4f}")

Try implementing confidence-weighted voting where each model's vote is weighted by its prediction confidence:

In [ ]:
# Complete this: Confidence-weighted ensemble
# Instead of simple majority voting, weight each vote by confidence

# Get prediction probabilities from all models
all_probabilities = []

for clf, test_emb in classifiers:
    # Hint: Use predict_proba() to get probabilities instead of predict()
    probs = None  # get probabilities for each prediction
    all_probabilities.append(probs)

all_probabilities = np.array(all_probabilities) if all_probabilities[0] is not None else None

# Calculate weighted predictions
weighted_predictions = []

for i in range(len(test_subset)):
    # For each sample, sum the confidence-weighted votes
    # Hint: For each class, sum probabilities across all models using all_probabilities[:, i, :]
    # Then use np.sum() along axis 0 to get scores for each class
    class_scores = None  # calculate weighted scores for each class
    
    # Pick class with highest weighted score
    # Hint: Use np.argmax()
    weighted_pred = None  # argmax of class_scores
    weighted_predictions.append(weighted_pred if weighted_pred is not None else 0)

weighted_f1 = f1_score(test_subset["label"], weighted_predictions, average='weighted')

print(f"\nConfidence-weighted ensemble F1: {weighted_f1:.4f}")
print(f"Simple majority F1: {ensemble_f1:.4f}")
print(f"Difference: {weighted_f1 - ensemble_f1:+.4f}")

#### Hard Task 4: Cross-Domain Transfer Learning

Can a classifier trained on movie reviews work on restaurant, product, or book reviews? Observe zero-shot transfer (no adaptation) performance on each domain, see which domains transfer well and which don't, then try few-shot adaptation (adding just 4 examples from target domain). Analyze domain similarity using embedding distances and optionally add your own custom domain to test.

Load dependencies and set up source/target domains:

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

Define source domain (movies) and target domains:

In [ ]:
movie_data = load_dataset("rotten_tomatoes")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

movie_train = movie_data["train"].shuffle(seed=42).select(range(2000))
movie_test = movie_data["test"].shuffle(seed=42).select(range(200))

In [ ]:
restaurant_reviews = {
    'text': [
        "Amazing food and excellent service!",
        "Best restaurant in town, highly recommend",
        "Delicious meals and great atmosphere",
        "Outstanding cuisine and friendly staff",
        "Terrible food, very disappointing",
        "Awful service and poor quality",
        "Not worth the money, mediocre at best",
        "Disgusting food and rude waiters",
        "The pasta was okay but nothing special",
        "Decent place for a quick meal"
    ],
    'label': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
}

product_reviews = {
    'text': [
        "This product is amazing! Works perfectly",
        "Excellent quality, very satisfied",
        "Great value for money, highly recommend",
        "Perfect! Exactly what I needed",
        "Terrible product, broke immediately",
        "Waste of money, very poor quality",
        "Doesn't work as advertised, disappointed",
        "Awful, don't buy this",
        "It's okay, does the job",
        "Average product, nothing special"
    ],
    'label': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
}

book_reviews = {
    'text': [
        "Brilliant book! Couldn't put it down",
        "Masterfully written, highly engaging",
        "One of the best books I've read",
        "Fantastic story and great characters",
        "Boring and poorly written",
        "Terrible book, waste of time",
        "Disappointing, not worth reading",
        "Awful plot and weak characters",
        "Decent read but nothing groundbreaking",
        "It was fine, not great not terrible"
    ],
    'label': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
}

Train classifier on source domain (movies):

In [ ]:
print("Training on movie reviews...")
train_embeddings = model.encode(movie_train["text"], show_progress_bar=True)

clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(train_embeddings, movie_train["label"])

# Test on source domain
movie_test_embeddings = model.encode(movie_test["text"], show_progress_bar=False)
movie_test_pred = clf.predict(movie_test_embeddings)
source_f1 = f1_score(movie_test["label"], movie_test_pred, average='weighted')

print(f"\nSource domain F1: {source_f1:.4f}")

Test zero-shot transfer to target domains (no adaptation):

In [ ]:
target_domains = {
    "Restaurant": restaurant_reviews,
    "Product": product_reviews,
    "Book": book_reviews,
}

transfer_results = {}

print("\nZero-shot transfer results:")
for domain_name, domain_data in target_domains.items():
    domain_embeddings = model.encode(domain_data['text'])
    domain_pred = clf.predict(domain_embeddings)
    domain_f1 = f1_score(domain_data['label'], domain_pred, average='weighted')
    domain_acc = accuracy_score(domain_data['label'], domain_pred)

    print(f"\n{domain_name}: F1={domain_f1:.4f}, Acc={domain_acc:.4f}")
    print(f"  Drop from source: {source_f1 - domain_f1:.4f}")

    transfer_results[domain_name] = {
        'zero_shot_f1': domain_f1,
        'embeddings': domain_embeddings
    }

Few-shot adaptation (add 4 examples from target domain):

In [ ]:
adaptation_size = 4

# Experiment: Try different adaptation sizes
# Uncomment to see how performance changes with more examples:
# adaptation_size = 2   # Minimal adaptation
# adaptation_size = 8   # More examples
# adaptation_size = 16  # Even more examples

print(f"\nFew-shot adaptation ({adaptation_size} examples):")
for domain_name, domain_data in target_domains.items():
    # Split: first N for adaptation, rest for testing
    adapt_texts = domain_data['text'][:adaptation_size]
    adapt_labels = domain_data['label'][:adaptation_size]
    test_texts = domain_data['text'][adaptation_size:]
    test_labels = domain_data['label'][adaptation_size:]

    # Combine source + adaptation
    adapt_embeddings = model.encode(adapt_texts)
    combined_embeddings = np.vstack([train_embeddings, adapt_embeddings])
    combined_labels = list(movie_train["label"]) + adapt_labels

    # Retrain
    clf_adapted = LogisticRegression(random_state=42, max_iter=1000)
    clf_adapted.fit(combined_embeddings, combined_labels)

    # Test
    test_embeddings = model.encode(test_texts)
    adapted_pred = clf_adapted.predict(test_embeddings)
    adapted_f1 = f1_score(test_labels, adapted_pred, average='weighted')

    zero_shot_f1 = transfer_results[domain_name]['zero_shot_f1']
    improvement = adapted_f1 - zero_shot_f1

    print(f"\n{domain_name}:")
    print(f"  Zero-shot: {zero_shot_f1:.4f}")
    print(f"  Adapted:   {adapted_f1:.4f}")
    print(f"  Gain:      {improvement:+.4f}")

    transfer_results[domain_name]['adapted_f1'] = adapted_f1
    transfer_results[domain_name]['improvement'] = improvement

Analyze domain similarity (embedding distance):

In [ ]:
source_centroid = np.mean(train_embeddings, axis=0)

print("\nDomain distances from source:")
for domain_name in target_domains.keys():
    domain_embeddings = transfer_results[domain_name]['embeddings']
    domain_centroid = np.mean(domain_embeddings, axis=0)
    distance = np.linalg.norm(source_centroid - domain_centroid)

    zero_f1 = transfer_results[domain_name]['zero_shot_f1']
    drop = source_f1 - zero_f1

    print(f"\n{domain_name:12s}: distance={distance:.3f}, drop={drop:.3f}")

print("\nObservation: Smaller distance = better transfer")

### Questions

1. Which domain transferred best from movies? Which worst?

2. Do you see patterns in what transfers well vs what fails?

3. After few-shot adaptation: Which domains benefited most from just 4 examples?
